In [2]:
import numpy as np
import json
import time
import sys
import redis
import msgpack
from multiprocessing import Process, Manager

In [3]:
# Random array of floats sized 1858
pool = redis.ConnectionPool(host='localhost', port=6379, db=0)
#r = redis.Redis(connection_pool=pool)
fake_policy = np.random.rand(1858).astype(np.float32)
fake_value = np.random.rand(1).astype(np.float32)
fake_policy, fake_value

(array([0.56664246, 0.5617776 , 0.9995359 , ..., 0.7664546 , 0.9325946 ,
        0.69126344], dtype=float32),
 array([0.91439945], dtype=float32))

In [4]:
times = []
for i in range(100):
    t1 = time.time()
    fake_policy_quantized = np.round(fake_policy * 255).astype(np.uint16)
    json_str = json.dumps({
        "policy": fake_policy_quantized.tolist(),
        "value": fake_value.tolist()
    })
    r.set("test", json_str)
    t2 = time.time()
    times.append(t2 - t1)
print("Average time to serialize: ", np.mean(times))

Average time to serialize:  0.00033260107040405273


In [4]:
times = []
for i in range(100):
    t1 = time.time()
    json_redis = r.get("test")
    data = json.loads(json_redis)
    fake_policy_out = np.array(data["policy"], dtype=np.uint16).astype(np.float32) / 255
    fake_value_out = np.array(data["value"], dtype=np.float32)
    t2 = time.time()
    times.append(t2 - t1)
print("Average time to deserialize: ", np.mean(times))

Average time to deserialize:  0.0002930087804794312


In [5]:
sys.getsizeof(fake_policy), sys.getsizeof(json_str), sys.getsizeof(json_redis), sys.getsizeof(data), sys.getsizeof(fake_policy_out)

(7544, 8600, 8584, 232, 7544)

In [26]:
times = []
for i in range(100):
    t1 = time.time()
    fake_policy_quantized = np.round(fake_policy * 255).astype(np.uint16)
    package = msgpack.dumps({
        "policy": fake_policy.tolist(),
        "value": fake_value.tolist()
    })
    r.set("test2", package)
    t2 = time.time()
    times.append(t2 - t1)
print("Average time to serialize: ", np.mean(times))

Average time to serialize:  0.00032272100448608396


In [27]:
times = []
for i in range(100):
    t1 = time.time()
    package_redis = r.get("test2")
    data = msgpack.loads(package_redis)
    fake_policy_out = np.array(data["policy"], dtype=np.uint16).astype(np.float32) / 255
    fake_value_out = np.array(data["value"], dtype=np.float32)
    t2 = time.time()
    times.append(t2 - t1)
print("Average time to deserialize: ", np.mean(times))

Average time to deserialize:  0.00036874055862426755


In [25]:
sys.getsizeof(fake_policy), sys.getsizeof(package), sys.getsizeof(package_redis), sys.getsizeof(data), sys.getsizeof(fake_policy_out)

(7544, 16782, 2856, 232, 7544)

In [11]:
fake_policy_out, fake_value_out

(array([0.9254902 , 0.13333334, 0.9137255 , ..., 0.84313726, 0.1764706 ,
        0.7764706 ], dtype=float32),
 array([0.44071597], dtype=float32))

In [15]:

def test_write(cache, pool):
    r = redis.Redis(connection_pool=pool)
    times = []
    for i in range(100):
        t1 = time.time()
        fake_policy_quantized = np.round(fake_policy * 255).astype(np.uint16)
        json_str = json.dumps({
            "policy": fake_policy_quantized.tolist(),
            "value": fake_value.tolist()
        })
        r.set("test", json_str)
        t2 = time.time()
        times.append(t2 - t1)
    print("Average time to serialize: ", np.mean(times))

def test_read(cache, pool):
    r = redis.Redis(connection_pool=pool)
    times = []
    for i in range(100):
        t1 = time.time()
        package_redis = r.get("test2")
        data = msgpack.loads(package_redis)
        fake_policy_out = np.array(data["policy"], dtype=np.uint16).astype(np.float32) / 255
        fake_value_out = np.array(data["value"], dtype=np.float32)
        t2 = time.time()
        times.append(t2 - t1)
    print("Average time to deserialize: ", np.mean(times))

with Manager() as manager:
    cache = manager.dict()
    p1 = Process(target=test_write, args=(cache, pool))
    p2 = Process(target=test_read, args=(cache, pool))
    p1.start()
    p2.start()
    p1.join()
    p2.join()
    print(len(cache), sys.getsizeof(cache))

Average time to serialize: Average time to deserialize:   0.000284445285797119130.00037267684936523435

0 48


In [19]:
r = redis.Redis(connection_pool=pool)
times = []
for i in range(100):
    t1 = time.time()
    n = r.get("test6")
    t2 = time.time()
    times.append(t2 - t1)
print("Average time to fetch None: ", np.mean(times))

Average time to fetch None:  7.092714309692383e-05


In [1]:
from selfplay import play_game
from utils import FakeTRTFunc
from configs import selfplayConfig


trt_func = FakeTRTFunc()
_, _, _ = play_game(trt_func, 2)


2025-01-11 22:39:59.542684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.



  +---+---+---+---+---+---+---+---+
  |   |   |   | k |   |   | n | b | 8
  +---+---+---+---+---+---+---+---+
  |   |   | r |   |   | P |   |   | 7
  +---+---+---+---+---+---+---+---+
  |   | R | R |   |   |   |   |   | 6
  +---+---+---+---+---+---+---+---+
  | p |   |   |   |   | P | P |   | 5
  +---+---+---+---+---+---+---+---+
  | P |   | P |   | N | K |   |   | 4
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   | q |   | r | 3
  +---+---+---+---+---+---+---+---+
  |   |   |   | P |   |   |   |   | 2
  +---+---+---+---+---+---+---+---+
  |   |   |   |   |   |   |   |   | 1
  +---+---+---+---+---+---+---+---+
    a   b   c   d   e   f   g   h

Fen: 3k2nb/2r2P2/1RR5/p4PP1/P1P1NK2/5q1r/3P4/8 w - - 1 62
Key: 70d23d86cd206d18
Black wins by checkmate in 122 moves in 14.10 seconds


In [14]:
import time
images_np = np.array(images, dtype=np.int64)
statistics_np = np.array(statistics, dtype=np.float32)
values_np = np.array(values, dtype=np.int64)

# Measure time to save the whole array
start_time = time.time()
np.savez('arrays.npz', images=images_np, statistics=statistics_np, values=values_np)
whole_save_time = time.time() - start_time

print(f"Time to save the whole array: {whole_save_time:.6f} seconds")

# Measure time to save each element from all three arrays in a loop
start_time = time.time()
for i in range(len(images_np)):
    np.savez(f'array_{i}.npy', images=images_np[i], statistics=statistics_np[i], values=values_np[i])
loop_save_time = time.time() - start_time

print(f"Time to save each element from all three arrays in a loop: {loop_save_time:.6f} seconds")

Time to save the whole array: 0.007873 seconds
Time to save each element from all three arrays in a loop: 0.088979 seconds


In [17]:
import numpy as np

neki = []
for i in range(10):
    neki.append(np.array([1, 2, 3, 4, 5], dtype=np.int64))

neki = np.array(neki, dtype=np.int64)
neki.shape

(10, 5)

In [ ]:

play_game